# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/internet_topology_cogentco.gml")

In [3]:
print(nx.info(g))

Graph with 197 nodes and 243 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 197 nodes and 243 edges
Graph with 197 nodes and 243 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/internet_topology_cogentco-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/internet_topology_cogentco-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/internet_topology_cogentco-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/internet_topology_cogentco-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

197

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[103,
 81,
 56,
 10,
 3,
 4,
 175,
 128,
 64,
 77,
 78,
 182,
 63,
 112,
 41,
 34,
 37,
 23,
 30,
 111,
 129,
 12,
 13,
 14,
 17,
 18,
 151,
 166,
 167,
 125,
 72,
 74,
 79,
 80,
 173,
 60,
 104,
 110,
 176,
 184,
 188,
 47,
 48,
 49,
 51,
 53,
 96,
 97,
 98,
 36,
 115,
 117,
 120,
 91,
 94,
 95,
 1,
 7,
 20,
 136,
 143,
 144,
 145,
 92,
 31,
 70,
 67,
 66,
 16,
 19,
 100,
 24,
 29,
 178,
 121,
 26,
 27,
 25,
 160,
 90,
 132,
 134,
 135,
 11,
 15,
 148,
 149,
 150,
 169,
 170,
 126,
 127,
 71,
 73,
 76,
 180,
 181,
 59,
 61,
 8,
 9,
 75,
 102,
 105,
 106,
 107,
 108,
 109,
 113,
 186,
 187,
 131,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 38,
 39,
 40,
 42,
 43,
 45,
 46,
 50,
 114,
 52,
 35,
 118,
 89,
 0,
 2,
 5,
 6,
 152,
 153,
 154,
 155,
 156,
 157,
 163,
 164,
 137,
 139,
 140,
 141,
 142,
 146,
 147,
 65,
 196,
 133,
 165,
 69,
 171,
 68,
 185,
 190,
 191,
 99,
 101,
 21,
 22,
 28,
 174,
 119,
 122,
 62,
 33,
 54,
 159,
 93,
 158,
 130,
 168,
 124,
 172,
 183,
 189,
 44,
 116,
 194,
 1

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{103: 197,
 81: 196,
 56: 195,
 10: 194,
 3: 193,
 4: 192,
 175: 191,
 128: 190,
 64: 189,
 77: 188,
 78: 187,
 182: 186,
 63: 185,
 112: 184,
 41: 183,
 34: 182,
 37: 181,
 23: 180,
 30: 179,
 111: 178,
 129: 177,
 12: 176,
 13: 175,
 14: 174,
 17: 173,
 18: 172,
 151: 171,
 166: 170,
 167: 169,
 125: 168,
 72: 167,
 74: 166,
 79: 165,
 80: 164,
 173: 163,
 60: 162,
 104: 161,
 110: 160,
 176: 159,
 184: 158,
 188: 157,
 47: 156,
 48: 155,
 49: 154,
 51: 153,
 53: 152,
 96: 151,
 97: 150,
 98: 149,
 36: 148,
 115: 147,
 117: 146,
 120: 145,
 91: 144,
 94: 143,
 95: 142,
 1: 141,
 7: 140,
 20: 139,
 136: 138,
 143: 137,
 144: 136,
 145: 135,
 92: 134,
 31: 133,
 70: 132,
 67: 131,
 66: 130,
 16: 129,
 19: 128,
 100: 127,
 24: 126,
 29: 125,
 178: 124,
 121: 123,
 26: 122,
 27: 121,
 25: 120,
 160: 119,
 90: 118,
 132: 117,
 134: 116,
 135: 115,
 11: 114,
 15: 113,
 148: 112,
 149: 111,
 150: 110,
 169: 109,
 170: 108,
 126: 107,
 127: 106,
 71: 105,
 73: 104,
 76: 103,
 180: 102,
 181:

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/Infra-InternetTopCogentco/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/Infra-InternetTopCogentco/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True